<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/6_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
from pptx import Presentation
from pptx.util import Inches

df = pd.DataFrame({
    'chart_name': ['chart_A', 'chart_B', 'chart_C', 'chart_D'],
    'base_on': ['LOT', 'WAFER', 'WAFER', 'WAFER'],
    'stat': ['MEAN', 'MEAN', 'MEAN', 'SIGMA']
})

prs = Presentation()

# 封面頁
slide_start = prs.slides.add_slide(prs.slide_layouts[0])
slide_start.shapes.title.text = 'review'
slide_start.placeholders[1].text = '2025-01-02'

# 圖片位置對應
img_positions = {
    'CPK': (Inches(0.5), Inches(1.5)),
    'MEAN_SHIFT': (Inches(5.0), Inches(1.5)),
    'SIGMA_RATIO': (Inches(0.5), Inches(4.5)),
    'TREND_CHART': (Inches(5.0), Inches(4.5)),
}

img_width = Inches(4.5)
img_height = Inches(3)

# === 🧠 預先掃描所有資料夾的圖片檔案 ===
# {(base_on, stat): [filename1, filename2, ...]}
folder_image_map = {}

# 所有組合
combinations = df[['base_on', 'stat']].drop_duplicates()

for _, row in combinations.iterrows():
    base_on, stat = row['base_on'], row['stat']
    folder = f"../../data_source/month/{base_on}/{stat}"
    if not os.path.exists(folder):
        print(f"⚠️ 路徑不存在：{folder}")
        folder_image_map[(base_on, stat)] = []
        continue
    # 儲存此組合下所有圖片檔名（不含路徑）
    folder_image_map[(base_on, stat)] = [
        f for f in os.listdir(folder)
        if f.endswith('.png') and '___' in f
    ]

# === 🖼 繪製每個 chart 的圖片頁面 ===
for _, chart in df.iterrows():
    chart_name = chart['chart_name']
    base_on = chart['base_on']
    stat = chart['stat']
    folder = f"../../data_source/month/{base_on}/{stat}"
    image_files = folder_image_map.get((base_on, stat), [])

    slide = prs.slides.add_slide(prs.slide_layouts[7])
    slide.shapes.title.text = chart_name

    # 篩出該 chart_name 開頭的圖片
    chart_images = [f for f in image_files if f.startswith(chart_name)]

    for img_file in chart_images:
        try:
            file_key = img_file[:-4].split('___')[1]
            if file_key not in img_positions:
                continue
            left, top = img_positions[file_key]
            img_path = os.path.join(folder, img_file)
            slide.shapes.add_picture(img_path, left, top, width=img_width, height=img_height)
        except Exception as e:
            print(f"❌ 無法處理圖片 {img_file}：{e}")

prs.save('output.pptx')


## 2. monthly callback


✅ 解法 1：儲存 session-id 到 dcc.Store(persistence=True)

In [ ]:
# 這會讓使用者在 刷新頁面或誤關閉後重新開啟頁面時，仍保留原來的 session-id。
# (這僅對「同一個使用者」瀏覽器刷新有效，若換裝置登入就是新 session。)
dcc.Store(id="session-id", data=str(uuid.uuid4()), storage_type='session', persistence=True)

✅ 解法 2：讓 update_ui / kill_process 自動 fallback

(若抓不到 session_id，自動切換成原本只有按下report狀態)
## 注意! 被kill 會沒有寄信

In [ ]:
## 原本 : progress = progress_pool.get(session_id, default_progress())
if session_id not in progress_pool:
    return [html.Div("⚠️ 尚未啟動流程")]  # 或 disable interval
